<a href="https://colab.research.google.com/github/vizzies/grcml-challenge-a-starter/blob/main/starter-notebooks/grcml_challenge_a_shruti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Uncomment the below lines if these dependencies are not already installed
!pip install --upgrade scispacy
!pip install --upgrade spacy
!pip install --upgrade gensim
!pip install --upgrade pyLDAvis

# Uncomment the below line if en_core_sci_sm is not loaded on your system.
!pip install --upgrade https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_sm-0.2.4.tar.gz

#uncomment if you need to read the data files from google cloud storage
#from google.cloud import storage

#uncomment if you need to read data in from Google Drive that's mounted to a CoLab notebook
from google.colab import drive

import time
import bokeh
from sklearn.manifold import TSNE
import pandas as pd
import scispacy
import spacy
import string
from spacy.lang.en import English
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
from gensim import similarities
from sklearn.neighbors import NearestNeighbors
from nltk.corpus import stopwords  
from nltk.stem.snowball import SnowballStemmer
from scipy.spatial.distance import jensenshannon
import numpy as np
from google.cloud import storage

#nlp = spacy.load('en_core_sci_sm')
import en_core_sci_sm
nlp = en_core_sci_sm.load()

import nltk
# Uncomment the below line if nltk's stopwords are not already on your system.
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 
stemmer = SnowballStemmer('english')
nltk.download('wordnet')

from bokeh.models import HoverTool, CustomJS, ColumnDataSource
from bokeh.palettes import viridis, Viridis256, magma, Turbo256, linear_palette
from bokeh.transform import factor_cmap
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

In [3]:
import pandas as pd
# df = pd.read_csv("/content/drive/My Drive/NASA/grc-ml-hackathon/challenge-ab/processed/Combined_Output.txt", sep=" ", header=None)
# df.columns = ["a", "b", "c", "etc."]
# df
data = open("/content/drive/My Drive/NASA/grc-ml-hackathon/challenge-ab/processed/Combined_Output.txt", "r")
print(data.read())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
with open("/content/drive/My Drive/NASA/grc-ml-hackathon/challenge-ab/processed/Combined_Output.txt", "r") as infile:
    data = infile.readlines()
    
print("\n\n".join((data[0][:500], data[1][:500], data[2][:500], data[25][:5000])))

In [ ]:
print(data)

In [ ]:
import urllib.request
!pip install pdfminer
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
from io import StringIO

In [ ]:
# for d in data:
#   pdfs = data[data[d].str.endswith('.pdf')]
#   pdfs[d] = pdf_links.apply(parse_text, axis=1)

In [7]:
import re
import nltk
def clean_html(sentence):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def clean_punc(word):
    cleaned = re.sub(r'[?|!|\'|#]', r'', word)
    cleaned = re.sub(r'[.|,|)|(|\|/]', r' ', cleaned)
    return cleaned

In [8]:
# Lemmatization
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def word_lemmatizer(text):
  lem = [lemmatizer.lemmatize(i) for i in text]
  return lem

In [19]:
import unicodedata
TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    return TAG_RE.sub('', text)
def preprocess_text(sen):
    sentence = str(sen)
    # Removing html tags
    sentence = remove_tags(sentence)
    # Remove hyphenation if at the end of a line
    sentence = sentence.replace('-\n', '')
    # Fix ligatures
    sentence = unicodedata.normalize("NFKD", sentence)
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    return sentence

In [ ]:
# Cleaning html tags with Beautiful Soup
import bs4 as bs
def clean_html(text):
    soup = bs(text, 'lxml')
    html_cleaned = soup.get_text()
    return html_cleaned

In [ ]:
import string
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
def stopwords(text):
  stop = stopwords.words('english')
  sno = SnowballStemmer('english')
  sno.stem(text)

In [ ]:
word_lemmatizer(data)
#clean_html(data)
print("\n\n".join((data[0][:500], data[1][:500], data[2][:500], data[25][:5000])))

In [ ]:
preprocess_text(data)
print("\n\n".join((data[0][:500], data[1][:500], data[2][:500], data[25][:5000])))

tf-idf

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from functools import reduce
from math import log
import re
import string

In [ ]:
# Clearing the data and tokenizing
l_A = data[0].lower().split()
l_B = data[1].lower().split()
l_C = data[2].lower().split()

print(l_A)
print(l_B)
print(l_C)

In [ ]:
## Bag of Words calculation
word_set = set(l_A).union(set(l_B)).union(set(l_C))
print(word_set)

In [25]:
## Continuing Bag of Words calculation

word_dict_A = dict.fromkeys(word_set, 0)
word_dict_B = dict.fromkeys(word_set, 0)
word_dict_C = dict.fromkeys(word_set, 0)

for word in l_A:
    word_dict_A[word] += 1

for word in l_B:
    word_dict_B[word] += 1

for word in l_C:
    word_dict_C[word] += 1

In [26]:
pd.DataFrame([word_dict_A, word_dict_B, word_dict_C])

,refractive,0.01767,sure,conclusion,(1981),removed,"1.4,",diamond,operation,"(16),",prior,mance,mg;bia,s.v.,**,67.8,"lemouedda,",nism,paper.,r.,12cd,decreased,sciencedirect,"configurations,",peaks,0:11223n=1,is,symbol,purpose,ignate,homographic,he=hx140,0.3211,0.,7.,1:,1.6421,19.2,(99.8%,corresponding,...,engineering,commercial,brayton,properties,general,good,sim-,imperfections,17871803.,thickness,cut,cific,governs,"are,",acknowledgements,pattern,"jeng,",refer,meter,#e=#b,cal,"filter,",tb,challenge,#5,v;x,"therefore,",16.3,"study),",internal,0.02632,dehumidifier.,strated,parts,assumes,"moscow,","lpiv,",www.elsevier,augmentation,all-russian
0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,79,1,1,1,11,2,1,1,1,4,1,0,0,37,...,1,0,0,0,18,2,0,0,0,6,0,1,0,1,0,0,0,12,0,1,3,0,0,0,0,2,4,0,0,2,1,0,0,0,1,1,0,0,2,1
1,1,0,3,5,0,1,0,28,0,0,1,2,0,1,0,1,1,1,0,1,1,3,1,1,0,0,118,0,0,0,0,0,0,0,2,1,0,1,1,1,...,0,3,0,2,2,0,0,1,1,0,1,0,0,0,3,3,1,0,1,0,0,1,0,0,4,0,0,1,0,2,0,1,1,1,0,0,1,1,0,0
2,0,0,0,4,0,0,1,1,1,0,1,3,0,0,0,0,1,0,0,1,0,0,1,0,1,0,107,0,0,0,0,0,0,0,3,0,0,0,1,3,...,1,0,1,0,1,1,2,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0


In [27]:
def compute_tf(word_dict, l):
    tf = {}
    sum_nk = len(l)
    for word, count in word_dict.items():
        tf[word] = count/sum_nk
    return tf

def compute_idf(strings_list):
    n = len(strings_list)
    idf = dict.fromkeys(strings_list[0].keys(), 0)
    for l in strings_list:
        for word, count in l.items():
            if count > 0:
                idf[word] += 1
    
    for word, v in idf.items():
        idf[word] = log(n / float(v))
    return idf

In [28]:
tf_A = compute_tf(word_dict_A, l_A)
tf_B = compute_tf(word_dict_B, l_B)
tf_C = compute_tf(word_dict_C, l_C)

idf = compute_idf([word_dict_A, word_dict_B, word_dict_C])

In [29]:
def compute_tf_idf(tf, idf):
    tf_idf = dict.fromkeys(tf.keys(), 0)
    for word, v in tf.items():
        tf_idf[word] = v * idf[word]
    return tf_idf

In [30]:
tf_idf_A = compute_tf_idf(tf_A, idf)
tf_idf_B = compute_tf_idf(tf_B, idf)
tf_idf_C = compute_tf_idf(tf_C, idf)

pd.DataFrame([tf_idf_A, tf_idf_B, tf_idf_C])

,refractive,0.01767,sure,conclusion,(1981),removed,"1.4,",diamond,operation,"(16),",prior,mance,mg;bia,s.v.,**,67.8,"lemouedda,",nism,paper.,r.,12cd,decreased,sciencedirect,"configurations,",peaks,0:11223n=1,is,symbol,purpose,ignate,homographic,he=hx140,0.3211,0.,7.,1:,1.6421,19.2,(99.8%,corresponding,...,engineering,commercial,brayton,properties,general,good,sim-,imperfections,17871803.,thickness,cut,cific,governs,"are,",acknowledgements,pattern,"jeng,",refer,meter,#e=#b,cal,"filter,",tb,challenge,#5,v;x,"therefore,",16.3,"study),",internal,0.02632,dehumidifier.,strated,parts,assumes,"moscow,","lpiv,",www.elsevier,augmentation,all-russian
0,0.000000,0.000093,0.000000,0.000000,0.000093,0.000000,0.000000,0.000000,0.000000,0.000093,0.000000,0.000000,0.000093,0.000000,0.000093,0.000000,0.000000,0.000000,0.000093,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000093,0.0,0.000093,0.000093,0.000093,0.001021,0.000186,0.000093,0.000093,0.0,0.000137,0.000093,0.000000,0.000000,0.0,...,0.000034,0.000000,0.000000,0.000000,0.0,0.000068,0.000000,0.000000,0.000000,0.000557,0.000000,0.000093,0.000000,0.000093,0.000000,0.000000,0.000000,0.001113,0.000000,0.000093,0.000278,0.000000,0.000000,0.000000,0.000000,0.000186,0.000371,0.000000,0.000000,0.000068,0.000093,0.000000,0.000000,0.000000,0.000093,0.000093,0.000000,0.000000,0.000186,0.000093
1,0.000111,0.000000,0.000333,0.000205,0.000000,0.000111,0.000000,0.001148,0.000000,0.000000,0.000041,0.000082,0.000000,0.000111,0.000000,0.000111,0.000041,0.000111,0.000000,0.000041,0.000111,0.000333,0.000041,0.000111,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000041,0.000000,0.000111,0.000041,0.0,...,0.000000,0.000333,0.000000,0.000222,0.0,0.000000,0.000000,0.000111,0.000041,0.000000,0.000041,0.000000,0.000000,0.000000,0.000333,0.000333,0.000041,0.000000,0.000111,0.000000,0.000000,0.000111,0.000000,0.000000,0.000444,0.000000,0.000000,0.000111,0.000000,0.000082,0.000000,0.000041,0.000111,0.000041,0.000000,0.000000,0.000111,0.000041,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000239,0.000000,0.000000,0.000162,0.000060,0.000162,0.000000,0.000060,0.000179,0.000000,0.000000,0.000000,0.000000,0.000060,0.000000,0.000000,0.000060,0.000000,0.000000,0.000060,0.000000,0.000162,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000060,0.0,...,0.000060,0.000000,0.000162,0.000000,0.0,0.000060,0.000324,0.000000,0.000060,0.000000,0.000060,0.000000,0.000162,0.000000,0.000000,0.000000,0.000060,0.000000,0.000000,0.000000,0.000000,0.000000,0.000162,0.000162,0.000000,0.000000,0.000000,0.000000,0.000162,0.000000,0.000000,0.000060,0.000000,0.000060,0.000000,0.000000,0.000000,0.000060,0.000000,0.000000


In [32]:
tfidf = TfidfVectorizer()
features = tfidf.fit_transform(data)

In [33]:
pd.DataFrame(features.todense(), columns = tfidf.get_feature_names())

,00,000,0000,00000,0000086,000010001001,0000162,0000214,00003,00005,00006,00008,0000e,0001,00018,0002,00025,00026,00028,0002d,0002x,0003,00030,000307,00033,000354,000376,0004,000403,00041,000447,000487,000495,0005,000504,00055,000567,0006,00066,0006x,...,zoove,zooword,zou,zr,zsioe,zslo,zsooe,zsse,zsurf,zt1atai,zt1e,zt_,ztci,ztest,ztp,ztutzu,ztz,zu,zubair,zuber,zubers,zuk,zukaukas,zukaus,zukauskas,zukauskus,zun,zuo,zur,zurich,zusammenfassung,zv,zvv,zwio0,zwo,zx,zxq,zxy,zygo,zz
0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.004161,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.001975,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.006225,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
106,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
107,0.000000,0.005812,0.0,0.0,0.0,0.003316,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003316,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
108,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
def get_entities(temp_article):
    '''
    A function for preprocessing an article's text.  Spacy's en_core_sci_sm library is used to identify scientfic domain terms. 
    Stopword removal and snowball stemming is then used to cleanup each of term.
    Arg:
        temp_article: the string input for preprocessing
    Returns:
        temp_entities: a list of preprocessed terms
    '''
    if len(temp_article) > nlp.max_length:
        nlp.max_length = len(temp_article)
    doc = nlp(temp_article)
    temp_entities = []
    for n in doc.ents:
        temp_str = ''
        for w in n:
            if w not in stop_words:
                if str(w).isalpha():
                    temp_str += stemmer.stem(str(w)) + ' '
                else:
                    temp_str += str(w) + ' '
        temp_entities.append(temp_str.strip())
    return temp_entities

In [36]:
bow_vector = CountVectorizer(tokenizer=get_entities, min_df=2, max_df=5, ngram_range=(1,3))
data_vectorized = bow_vector.fit_transform(data)

In [37]:
cluster_count = 10

# Build LDA Model
lda_model = LatentDirichletAllocation(n_components=cluster_count,               # Number of topics
                                      #doc_topic_prior=1.0,
                                      max_iter=100,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )

In [38]:
lda_output = lda_model.fit_transform(data_vectorized)

In [40]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, bow_vector, mds='tsne', sort_topics=False)
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
0      32.420952   8.115230       1        1   8.176317
1       0.648955  71.707100       2        1   8.952719
2      71.019684   9.971006       3        1   6.495085
3       3.785981  33.185135       4        1  10.630576
4     -42.666500  -9.222070       5        1   8.625929
5      -3.621689 -43.526516       6        1  17.255243
6      -3.632402  -4.171708       7        1   8.341987
7     -36.584400  35.804413       8        1   7.087792
8      44.391380  50.615383       9        1  12.360267
9      40.235809 -31.591665      10        1  12.074085, topic_info=                         Term        Freq  ...  logprob  loglift
58                        _ _  130.000000  ...  30.0000  30.0000
9                           -   99.000000  ...  29.0000  29.0000
12208               jet array   77.000000  ...  28.0000  28.0000
59                      _ _ _  105.000000  ...  27.0000  27.0000
4070                      cpf   89.000000  ...  26.0000  26.0000
...                       ...         ...  ...      ...      ...
15994             page intent   19.432308  ...  -6.3125   1.7361
9267           heat mass tran   18.624989  ...  -6.3549   1.6188
20046                 s/d s/d   16.206969  ...  -6.4940   1.6886
17590  plate-fin pin-fin heat   14.595399  ...  -6.5987   1.5186
14328      micro-pinfin array   18.625756  ...  -6.3549   0.8991

[517 rows x 6 columns], token_table=       Topic      Freq                                           Term
term                                                                 
9          4  0.870137                                              -
9          6  0.120019                                              -
10         4  0.879776                                            - -
10         6  0.117303                                            - -
11         4  0.881515                                          - - -
...      ...       ...                                            ...
25418      6  0.152396                                         z. wan
25418      9  0.152396                                         z. wan
25488     10  0.950844                    zhou intern journal of heat
25489     10  0.950826  zhou intern journal of heat mass transfer 136
25508      9  0.931640                                      zoom view

[926 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [23]:
top_cluster = []
for n in range(lda_output.shape[0]):
    categ = np.argwhere(lda_output[n,:] == np.amax(lda_output[n,:]))
    top_cluster.append(str(categ[0][0]+1))

In [24]:
test = data_vectorized.todense()

In [25]:
tsne = TSNE(random_state=2017, perplexity=30, n_iter=5000)#, early_exaggeration=120)
embedding = tsne.fit_transform(test)

In [26]:
unique_cats = list(set(top_cluster))
unique_cats_count = len(unique_cats)
print('There are ' + str(unique_cats_count) + ' unique categories in our dataset.')

There are 10 unique categories in our dataset.


In [27]:
viridis_spec = viridis(unique_cats_count)
magma_spec = magma(unique_cats_count)
turbo_spec = linear_palette(Turbo256, unique_cats_count)

In [28]:
source = ColumnDataSource(data=dict(
    x=embedding[:,0],
    y=embedding[:,1],
    top_cluster=top_cluster
))

TOOLTIPS = [
    ("index", '$index'),
    ("Title", '@desc'),
    ('Category', '@top_cluster')
]

In [29]:
p = figure(title="t-SNE Projection of Vectorized Heat Exchangers", tooltips=TOOLTIPS, plot_height=800, plot_width=800)
p.circle('x', 'y', size=10, source=source, fill_alpha=0.75, fill_color=factor_cmap('top_cluster', palette=magma_spec, factors=top_cluster))
show(p)

/usr/local/lib/python3.6/dist-packages/bokeh/models/mappers.py:147: UserWarning: Palette length does not match number of factors. ['7', '9', '6', '7', '4', '9', '5', '7', '6', '2', '5', '10', '9', '4', '1', '10', '7', '6', '2', '6', '4', '10', '6', '7', '6', '6', '10', '7', '3', '10', '5', '7', '9', '2', '10', '4', '7', '3', '2', '3', '2', '2', '9', '4', '4', '10', '4', '5', '7', '4', '5', '8', '10', '6', '7', '4', '7', '9', '8', '3', '9', '2', '9', '9', '1', '10', '3', '5', '4', '8', '8', '3', '5', '10', '8', '5', '3', '5', '7', '9', '2', '2', '8', '1', '6', '8', '6', '7', '10', '4', '1', '10', '6', '1', '6', '9', '9', '3', '7', '1'] will be assigned to `nan_color` gray
  warnings.warn("Palette length does not match number of factors. %s will be assigned to `nan_color` %s" % (extra_factors, self.nan_color))
